# Get Data for sp500

In [2]:
import importlib
import src.qp.data_loader as dl
importlib.reload(dl)

spx = dl.sp500_from_wikipedia()
tickers = spx["Code"].dropna().unique().tolist()
len(tickers), tickers[:10]


/Users/matijasarenac/code/quant-research-projects/src/qp/data_loader.py:86: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html.text)


(503, ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A'])

In [3]:
res = dl.update_prices_eod_for_list(tickers, exchange='US', start='2010-01-01')
sum(v is not None for v in res.values()), 'saved of', len(res)

(503, 'saved of', 503)

In [8]:
res.get('AAPL')

PosixPath('/Users/matijasarenac/code/quant-research-projects/data/processed/prices/AAPL.parquet')

In [11]:
import pandas as pd
df = pd.read_parquet('/Users/matijasarenac/code/quant-research-projects/data/processed/prices/AAPL.parquet')

In [12]:
df

,open,high,low,close,adj_close,volume
date,,,,,,
2010-01-04,213.4300,214.4996,212.3800,214.0096,6.4246,493729600
2010-01-05,214.6004,215.5888,213.2508,214.3792,6.4357,601904800
2010-01-06,214.3792,215.2304,210.7504,210.9688,6.3333,552160000
2010-01-07,211.7500,211.9992,209.0508,210.5796,6.3216,477131200
2010-01-08,210.2996,211.9992,209.0592,211.9796,6.3637,447610800
...,...,...,...,...,...,...
2025-10-08,256.5200,258.5200,256.1100,258.0600,258.0600,36496900
2025-10-09,257.8100,258.0000,253.1400,254.0400,254.0400,38322000
2025-10-10,254.9400,256.3800,244.0000,245.2700,245.2700,61999100


Works and all files saved

In [13]:
from pathlib import Path
import json

UNIVERSE_PATH = Path("data/processed/universe_sp500.json")

tickers = sorted([t for t, p in res.items() if p is not None])
UNIVERSE_PATH.parent.mkdir(parents=True, exist_ok=True)
UNIVERSE_PATH.write_text(json.dumps({"name": "S&P500", "asof": __import__("datetime").date.today().isoformat(), "tickers": tickers}, indent=2))

len(tickers), tickers[:10]


(503,
 ['A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM'])

In [14]:
from pathlib import Path
import json
import pandas as pd

PRICES_DIR = Path("data/processed/prices")
UNIVERSE_PATH = Path("data/processed/universe_sp500.json")

def load_universe(path: Path = UNIVERSE_PATH) -> list[str]:
    obj = json.loads(path.read_text())
    return obj["tickers"]

def load_price_one(ticker: str, cols=("adj_close",), dirpath: Path = PRICES_DIR) -> pd.DataFrame:
    """Loads a single ticker parquet/csv, returns a DateIndex df with requested cols."""
    p = dirpath / f"{ticker.upper()}.parquet"
    if p.exists():
        df = pd.read_parquet(p)
    else:
        p_csv = dirpath / f"{ticker.upper()}.csv"
        if not p_csv.exists():
            raise FileNotFoundError(f"No file for {ticker}")
        df = pd.read_csv(p_csv, parse_dates=["date"], index_col="date")
    df = df.sort_index()
    keep = [c for c in cols if c in df.columns]
    if not keep:
        raise ValueError(f"Requested cols {cols} not found in {ticker}")
    return df[keep]


In [ ]:
def build_panel_onecol(
    tickers: list[str],
    col: str = "adj_close",
    how: str = "outer",        # "inner" to keep common dates; "outer" to keep all
    start: str | None = None,
    end: str | None = None,
) -> pd.DataFrame:
    dfs = []
    for t in tickers:
        df = load_price_one(t, cols=(col,))
        if start: 
            df = df[df.index >= pd.to_datetime(start)]
        if end:   
            df = df[df.index <= pd.to_datetime(end)]
        df = df.rename(columns={col: t})
        dfs.append(df)
    panel = pd.concat(dfs, axis=1, join=how).sort_index()
    return panel

# Example:
tickers = load_universe()   # sample 50 to start, scale up when needed
px = build_panel_onecol(tickers, col="adj_close", how="outer", start="2015-01-01")
px.shape


(2712, 503)

In [21]:
px

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.1192,24.2610,42.0880,NaN,36.5806,18.5394,73.7724,72.340,44.4961,38.0018,...,23.7278,126.2672,25.7363,58.4088,33.2409,NaN,44.3734,100.2090,77.43,39.8874
2015-01-05,36.4236,23.5776,41.2959,NaN,36.5887,18.4284,72.5268,71.980,43.6869,36.6869,...,23.7278,124.1181,25.4442,56.8106,31.1721,NaN,43.4719,103.9472,76.34,39.6480
2015-01-06,35.8562,23.5798,41.0915,NaN,36.1732,18.4696,72.0037,70.530,42.6615,35.9637,...,23.7015,121.4706,25.5795,56.5086,30.9887,NaN,42.9383,103.0660,75.79,39.2612
2015-01-07,36.3321,23.9104,42.7523,NaN,36.4665,18.5774,73.5150,71.110,43.1101,36.5042,...,23.7738,125.0379,25.8004,57.0812,31.2332,NaN,44.3612,105.6293,77.72,40.0716
2015-01-08,37.4212,24.8291,43.1994,NaN,37.2160,18.9007,74.6360,72.915,43.8712,35.9710,...,24.0301,126.5422,26.0997,58.0313,31.4688,NaN,45.1340,106.7507,79.38,40.6887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-08,140.8100,258.0600,229.5849,119.99,134.2700,93.1000,251.3439,348.770,237.9300,62.2200,...,24.7000,121.5700,81.8500,114.0200,148.1900,81.11,146.0300,98.3600,307.28,143.4900
2025-10-09,140.1100,254.0400,229.0388,120.15,133.3100,92.2300,250.4100,347.470,237.8800,61.3100,...,24.4400,123.5500,81.2600,112.9100,146.8100,80.85,142.6900,97.3700,298.79,143.3900
2025-10-10,136.6200,245.2700,228.8502,118.19,132.5700,92.4700,240.9400,337.510,225.3200,60.8400,...,23.9300,119.8900,80.9300,110.7300,144.3200,74.67,141.6200,94.7100,275.89,141.1100
